In [1]:
import numpy as np

from tvm import relay
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing

In [308]:
bs = 1
num_class = 1000

img_shape = (3, 224, 224)

data_shape = (bs, ) + img_shape
output_shape = (bs, num_class)

In [309]:
mod, params = relay.testing.resnet.get_workload(
    num_layers=18, batch_size=bs, image_shape=img_shape
)

In [310]:
# set show_meta_data=True if you want to show meta data
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3), float32], %bn_data_beta: Tensor[(3), float32], %bn_data_moving_mean: Tensor[(3), float32], %bn_data_moving_var: Tensor[(3), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64), float32], %bn0_beta: Tensor[(64), float32], %bn0_moving_mean: Tensor[(64), float32], %bn0_moving_var: Tensor[(64), float32], %stage1_unit1_bn1_gamma: Tensor[(64), float32], %stage1_unit1_bn1_beta: Tensor[(64), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64), float32], %stage1_unit1_bn1_moving_var: Tensor[(64), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64), float32], %stage1_unit1_bn2_beta: Tensor[(64), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64), float32], %stage1_unit1_bn2_moving_var: Tensor[(64), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1

In [311]:
opt_level = 3
target = tvm.target.cuda()
with tvm.transform.PassContext(opt_level=opt_level):
    lib = relay.build(mod, target, params=params)
    # _, mylib, _ = relay.build(mod, target, params=params)


In [312]:
dev = tvm.cuda()
data = np.random.uniform(-1, 1, size=data_shape).astype("float32")
module = graph_executor.GraphModule(lib["default"](dev))

In [313]:
module.set_input("data", data)

In [314]:
module.run()

In [315]:
out = module.get_output(0, tvm.nd.empty(output_shape)).numpy()


In [316]:
print(out.flatten()[0:10])


[0.00089283 0.00103331 0.0009094  0.00102275 0.00108751 0.00106737
 0.00106262 0.00095838 0.00110792 0.00113151]


In [2]:
import tvm
import tensorflow as tf
import numpy as np
import tvm.relay as relay

2022-04-07 12:56:45.037977: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
model_keras = tf.keras.models.load_model('./very_simple_model.h5')

input_data = np.random.normal(0,1,(1,256,256,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


2022-04-07 12:56:45.825453: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-07 12:56:45.825575: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-07 12:56:45.861276: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-07 12:56:45.861865: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-04-07 12:56:45.861896: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-07 12:56:45.864408: I tensorflow/stream_executor/platfor

In [319]:
import numpy as np
import tvm
from tvm import te
import tvm.relay as relay

In [320]:
def example():
    shape = (1, 64, 54, 54)
    c_data = np.empty(shape).astype("float32")
    c = relay.const(c_data)
    weight = relay.var("weight", shape=(64, 64, 3, 3))
    x = relay.var("x", relay.TensorType((1, 64, 56, 56), "float32"))
    conv = relay.nn.conv2d(x, weight)
    y = relay.add(c, c)
    y = relay.multiply(y, relay.const(2, "float32"))
    y = relay.add(conv, y)
    z = relay.add(y, c)
    z1 = relay.add(y, c)
    z2 = relay.add(z, z1)
    return relay.Function([x, weight], z2)

In [321]:
# Let's first create a relay Module which contains one or multiple Relay
# functions for optimization.
f = example()
mod = tvm.IRModule.from_expr(f)

In [322]:
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) {
  %0 = add(meta[relay.Constant][0], meta[relay.Constant][0]);
  %1 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]);
  %2 = multiply(%0, 2f);
  %3 = add(%1, %2);
  %4 = add(%3, meta[relay.Constant][0]);
  %5 = add(%3, meta[relay.Constant][0]);
  add(%4, %5)
}

/* For debugging purposes the metadata section has been omitted.
 * If you would like to see the full metadata section you can set the 
 * option to `True` when invoking `astext`. 
 */


In [323]:
fold_const = relay.transform.FoldConstant()
# Then, we can invoke the pass on the given module. Note that the constant
# folding pass works at the function-level. That being said, each function in
# the module will be applied with the optimization. Users don't need to iterate
# through individual functions manually to apply this pass.
mod1 = fold_const(mod)
# We can see from the updated program that the constants are folded.
print(mod1)


def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) -> Tensor[(1, 64, 54, 54), float32] {
  %0 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %1 = add(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %2 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %3 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  add(%2, %3) /* ty=Tensor[(1, 64, 54, 54), float32] */
}




In [324]:
mod2 = relay.transform.EliminateCommonSubexpr()(mod1)
print(mod2)


def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) -> Tensor[(1, 64, 54, 54), float32] {
  %0 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %1 = add(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %2 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  add(%2, %2) /* ty=Tensor[(1, 64, 54, 54), float32] */
}




In [325]:
mod = relay.transform.FuseOps(fuse_opt_level=0)(mod)

# We can observe that the optimized module contains functions that only have
# a signle primitive op.
print(mod)

TVMError: Traceback (most recent call last):
  14: TVMFuncCall
  13: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::transform::Pass, tvm::IRModule)>::AssignTypedLambda<tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}>(tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&)
  12: tvm::transform::Pass::operator()(tvm::IRModule) const
  11: tvm::transform::Pass::operator()(tvm::IRModule, tvm::transform::PassContext const&) const
  10: tvm::relay::transform::FunctionPassNode::operator()(tvm::IRModule, tvm::transform::PassContext const&) const
  9: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedP

In [ ]:
relay.fromtext()

AttributeError: module 'tvm.relay' has no attribute 'fromtext'

In [ ]:
mod.import_from_std(mod1.astext())

In [4]:
import tvm
from tvm import relay
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *

In [ ]:
pattern = is_op("nn.relu")(is_op("nn.conv2d")(wildcard(), wildcard()))

# A graph.
x = relay.var('input')
w = relay.var('weight')
conv2d = relay.op.nn.conv2d(x, w)
relu = relay.op.nn.relu(conv2d)
print(relu)
# free_var %x: Tensor[(1, 3, 224, 224), float32]
# free_var %w: Tensor[(3, 3, 3, 3), float32]
# %0 = nn.conv2d(%x, %w, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 3, 222, 222), float32] */;
# free_var %b: Tensor[(3), float32]
# nn.bias_add(%0, %b) /* ty=Tensor[(1, 3, 222, 222), float32] */




free_var %input;
free_var %weight;
%0 = nn.conv2d(%input, %weight, padding=[0, 0, 0, 0]);
nn.relu(%0)


In [ ]:
# After partition.
print(pattern.partition(relu))
# free_var %x: Tensor[(1, 3, 224, 224), float32]
# free_var %w: Tensor[(3, 3, 3, 3), float32]
# free_var %b: Tensor[(3), float32]
# %1 = fn (%FunctionVar_0_0, %FunctionVar_0_1,
#          %FunctionVar_0_2, PartitionedFromPattern="nn.conv2d_nn.bias_add_") {
#   %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
#   nn.bias_add(%0, %FunctionVar_0_2)
# };
# %1(%x, %w, %b)

free_var %input;
free_var %weight;
%1 = fn (%FunctionVar_0_0, %FunctionVar_0_1, PartitionedFromPattern="nn.conv2d_nn.relu_") {
  %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
  nn.relu(%0)
};
%1(%input, %weight)


In [ ]:
print(pattern.partition(relu, {'Composite': 'one_layer'}))
# free_var %x: Tensor[(1, 3, 224, 224), float32]
# free_var %w: Tensor[(3, 3, 3, 3), float32]
# free_var %b: Tensor[(3), float32]
# %1 = fn (%FunctionVar_0_0, %FunctionVar_0_1,
#          %FunctionVar_0_2, Composite="one_layer",
#                            PartitionedFromPattern="nn.conv2d_nn.bias_add_") {
#   %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
#   nn.bias_add(%0, %FunctionVar_0_2)
# };
# %1(%x, %w, %b)

free_var %input;
free_var %weight;
%1 = fn (%FunctionVar_0_0, %FunctionVar_0_1, PartitionedFromPattern="nn.conv2d_nn.relu_", Composite="one_layer") {
  %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
  nn.relu(%0)
};
%1(%input, %weight)


In [ ]:
def check(pre):
    conv = pre.args[0]
    return (conv.attrs.data_layout == "NCHW") and bool(conv.checked_type.shape[0] == 1)

pattern.partition(relu, check=check)

ValueError: Traceback (most recent call last):
  7: TVMFuncCall
  6: tvm::runtime::TypedPackedFunc<tvm::RelayExpr (tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime::PackedFunc)>::AssignTypedLambda<tvm::relay::{lambda(tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime::PackedFunc)#3}>(tvm::relay::{lambda(tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime::PackedFunc)#3}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}::operator()(tvm::runtime::TVMArgs const, tvm::runtime::TVMRetValue) const
  5: tvm::relay::PartitionPattern(tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime

In [ ]:
class BatchnormCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        self.x = wildcard()
        self.var = wildcard()
        self.mean = wildcard()
        self.beta = wildcard()
        self.gamma = wildcard()
        self.eps = wildcard()

        self.pattern = self.gamma * (self.x - self.mean)/is_op("sqrt")(self.var + self.eps) + self.beta
        self.cnt = 0
        self.target = 2

    def callback(self, pre, post, node_map):
        if self.cnt != self.target:
            print(1)
            self.cnt += 1
        else :    
            # print(node_map)
            x = node_map[self.x][0]
            var = node_map[self.var][0]
            mean = node_map[self.mean][0]
            beta = node_map[self.beta][0]
            gamma = node_map[self.gamma][0]
            eps = node_map[self.eps][0]
            return relay.op.nn.batch_norm(x, gamma, beta, mean, var, epsilon = eps.data.numpy().item())[0]

# A graph of arithmetic operators that are functional equivalent to batch_norm.
x = relay.var('x')
var = relay.var('var')
mean = relay.var('mean')
beta = relay.var('beta')
gamma = relay.var('gamma')

JUNO = var + var + var + var + var + var + var 

BN = gamma * (x - mean)/relay.op.sqrt(var + var + relay.const(1e-5)) + beta
print(BN)
from tvm.relay.dataflow_pattern import rewrite
out = rewrite(BatchnormCallback(), BN)
# assert tvm.ir.structural_equal(out, relay.op.nn.batch_norm(x, gamma, beta, mean, var, epsilon = 1e-5)[0])

free_var %x;
free_var %mean;
free_var %gamma;
%0 = subtract(%x, %mean);
free_var %var;
%1 = add(%var, %var);
%2 = add(%1, 1e-05f);
%3 = multiply(%gamma, %0);
%4 = sqrt(%2);
%5 = divide(%3, %4);
free_var %beta;
add(%5, %beta)
1


TVMError: Traceback (most recent call last):
  10: TVMFuncCall
  9: tvm::runtime::TypedPackedFunc<tvm::RelayExpr (tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule)>::AssignTypedLambda<tvm::RelayExpr (*)(tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule)>(tvm::RelayExpr (*)(tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}::operator()(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*) const
  8: tvm::relay::RewritePatterns(tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule)
  7: tvm::relay::PatternRewriter::Rewrite(tvm::runtime::Array<tvm::relay::DFPatternCallback, void> const&, tvm::RelayExpr const&)
  6: tvm::relay::PatternGrouper::GroupMatches(tvm::relay::DFPattern const&, tvm:

In [6]:
class JUNOCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        # self.var = wildcard()
        # self.meta = wildcard()
        self.var = relay.var("var")
        self.meta = relay.var("meta")
        self.padding = [1, 1, 1, 1]
        self.channels = 32 
        self.kernel_size = [3, 3]
        self.out_dtype = "int32"
        # self.var = wildcard()
        
        # self.mean = wildcard()
        # self.beta = wildcard()
        # self.gamma = wildcard()
        # self.eps = wildcard()

        # self.pattern = self.gamma * (self.x - self.mean)/is_op("sqrt")(self.var + self.eps) + self.beta
        # self.pattern = is_op(relay.op.nn.max_pool2d)
        # self.pattern = relay.op.nn.nn.max_pool2d(self.var1)
        # self.pattern = relay.op.nn.conv2d(
        #     self.var, 
        #     self.meta, 
        #     padding=self.padding, 
        #     channels=self.channels,
        #     kernel_size=self.kernel_size,
        #     out_dtype=self.out_dtype
        # )
        # self.pattern = is_op('nn.conv2d')(wildcard(), wildcard(), is_constant(), is_constant(), is_constant())
        self.pattern = is_op('nn.bias_add')(self.var1, self.var2)
        # self.pattern = is_op('max_pool2d')(self.var1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0] )
        self.cnt = 0
        self.target = 0

    def callback(self, pre, post, node_map):
        var1 = node_map[self.var1][0]
        var2 = node_map[self.var2][0]
        original = relay.nn.bias_add(var1, var2)

        if self.cnt != self.target:
            print(self.cnt, self.target)
            print("fuck")
            self.cnt += 1
            return original
        else:
            self.cnt += 1
            # return var1 * var2
            original = relay.nn.bias_add(var1, var2)
            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(original, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            print(cast_to_int8)
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(cast_to_int8, dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return relay.op.annotation.stop_fusion(cast_to_float32)
            # return relay.cliprelay.right_shift(relay.add(relay.cast(
            #     relay.clip(
            #         relay.round(
            #             var1 * relay.const(16.0)
            #         ), 
            #         a_min=relay.const(-127.0), 
            #         a_max=relay.const(127.0)
            #     ), dtype="int8"), relay.const(512)), 10)

x1 = relay.var('x1')
x2 = relay.var('x2')
x3 = relay.var('x3')
x4 = relay.var('x4')
x5 = relay.var('x5')

JUNO = x1 + x2 + x3 + x4 + x5

print("===target relay===")
print(JUNO)
print("==================")
bc = JUNOCallback()
# print("target pattern")
# print(bc.pattern)
out = rewrite(JUNOCallback(), JUNO)
print("======OUT=========")
print(out)
print("==================")


===target relay===
free_var %x1;
free_var %x2;
%0 = add(%x1, %x2);
free_var %x3;
%1 = add(%0, %x3);
free_var %x4;
%2 = add(%1, %x4);
free_var %x5;
add(%2, %x5)
======OUT=========
free_var %x1;
free_var %x2;
%0 = add(%x1, %x2);
free_var %x3;
%1 = add(%0, %x3);
free_var %x4;
%2 = add(%1, %x4);
free_var %x5;
add(%2, %x5)


In [7]:
out = rewrite(JUNOCallback(), mod['main'])
# out_body = rewrite(JUNOCallback(), mod['main'].body)


free_var %input_1: Tensor[(1, 3, 256, 256), float32];
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32];
%0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
free_var %v_param_2: Tensor[(16), float32];
%1 = nn.bias_add(%0, %v_param_2);
%2 = multiply(%1, 16f);
%3 = round(%2);
%4 = clip(%3, a_min=-127f, a_max=127f);
cast(%4, dtype="int8")
1 0
fuck
2 0
fuck


In [8]:
out

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = multiply(%1, 16f);
  %3 = round(%2);
  %4 = clip(%3, a_min=-127f, a_max=127f);
  %5 = cast(%4, dtype="int8");
  %6 = cast(%5, dtype="int32");
  %7 = add(%6, 512);
  %8 = right_shift(%7, 10);
  %9 = clip(%8, a_min=-127f, a_max=127f);
  %10 = cast(%9, dtype="float32");
  %11 = annotation.stop_fusion(%10);
  %12 = nn.max_pool2d(%11, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %13 = nn.conv2d(%12, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %14 = nn.bias_add(%13, %v_param_4);
  %15 = nn.max_pool2d(%14, pool_size=[2, 2], strides=[

In [ ]:
mod['main'] = out
# mod['main'].body = out_body

In [ ]:
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

print(out)

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = multiply(%2, 16f);
  %4 = round(%3);
  %5 = clip(%4, a_min=-127f, a_max=127f);
  %6 = cast(%5, dtype="int8");
  %7 = cast(%6, dtype="int32");
  %8 = add(%7, 512);
  %9 = right_shift(%8, 10);
  %10 = clip(%9, a_min=-127f, a_max=127f);
  %11 = cast(%10, dtype="float32");
  %12 = nn.conv2d(%11, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %13 = nn.bias_add(%12, %v_param_4);
  %14 = nn.max_pool2d(%13, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %15 =

In [ ]:
mod['main'] = out

In [ ]:
# out_fun = tvm.IRModule.from_expr(out)
# mod['main'].body = out

target = 'cuda'
dev = tvm.cuda()

with tvm.transform.PassContext(opt_level=0):
    lib = relay.build(mod, target, params=params)


In [ ]:
mod['main']

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %6 = nn.conv2d(%5, %v_param_5, padding=[1, 1, 1, 1], channels=64, kernel_size=[3, 3]);
  %7 = nn.bias_add(%6, %v_param_6);
  nn.max_pool2d(%7, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0])
}

In [16]:
import json
import numpy as np
import pygraphviz as pgv

def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
            #A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(shape_size(json_data['attrs']['shape'][1][src[0]])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(shape_size(json_data['attrs']['shape'][1][node_idx])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')


In [ ]:
show_graph(lib['get_graph_json'](), "quanadd_0")


In [13]:
import tensorflow as tf
import numpy as np
model_keras = tf.keras.models.load_model('./very_simple_model.h5')

input_data = np.random.normal(0,1,(1,256,256,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


In [ ]:
import numpy as np

In [41]:
class JUNOCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var = wildcard()
        self.var2 = wildcard()
        self.pool_size = is_constant()
        self.strides = is_constant()
        self.padding = is_constant()
        # self.pool_size = wildcard()
        # self.strides = wildcard()
        # self.padding = wildcard()
        # self.var = wildcard()
        # self.meta = wildcard()
        # self.var = relay.var("var")
        # self.meta = relay.var("meta")
        self.padding = [1, 1, 1, 1]
        # self.channels = 32 
        # self.kernel_size = [3, 3]
        # self.out_dtype = "int32"
        # self.var = wildcard()
        
        # self.mean = wildcard()
        # self.beta = wildcard()
        # self.gamma = wildcard()
        # self.eps = wildcard()

        # self.pattern = self.gamma * (self.x - self.mean)/is_op("sqrt")(self.var + self.eps) + self.beta
        # self.pattern = is_op(relay.op.nn.max_pool2d)
        # self.pattern = relay.op.nn.nn.max_pool2d(self.var1)
        # self.pattern = relay.op.nn.conv2d(
        #     self.var, 
        #     self.meta, 
        #     padding=self.padding, 
        #     channels=self.channels,
        #     kernel_size=self.kernel_size,
        #     out_dtype=self.out_dtype
        # )
        # relay.nn.batch_norm()
        # self.pattern = is_op('nn.conv2d')(wildcard(), wildcard(), is_constant(), is_constant(), is_constant())
        # self.pattern = is_op('nn.bias_add')(self.var1, self.var2)
        self.pattern = is_op('nn.max_pool2d')(self.var)
        # self.pattern = is_op('nn.max_pool2d')(self.var, pool_size=self.pool_size, strides=self.strides, padding=self.padding)
        # op = is_op('nn.max_pool2d').has_attr({"pool_size" : self.pool_size, "strides":self.strides, "padding":self.padding})
        # op = is_op('nn.max_pool2d')
        # self.pattern = op(self.var)
        # self.pattern = relay.nn.max_pool2d(self.var, pool_size=self.pool_size, strides=self.strides, padding=self.padding)
        self.cnt = 0
        self.target = 0

    def callback(self, pre, post, node_map):
        # print("asd")
        # print(type(node_map[self.var][0]))
        # print("asd")
        var = node_map[self.var]
        
        # print(1)
        # print(var)
        # print(2)
        # original = relay.nn.max_pool2d(var, pool_size=(2, 2), strides=(2, 2), padding=(0, 0, 0, 0))
        original = post
        # pool_size = node_map[self.pool_size][0]
        # strides = node_map[self.strides][0]
        # padding = self.padding
        
        # padding = node_map[self.padding][0]
        # var2 = node_map[self.var2][0]
        # original = relay.nn.max_pool2d(var1, pool_size=(2, 2), strides=(2, 2), padding=(0, 0, 0, 0))

        if self.cnt != self.target:
            # print(self.cnt, self.target)
            print("pass")
            self.cnt += 1
            return post
        else:
            self.cnt += 1
            # return var1 * var2
            # original = relay.nn.bias_add(var1, var2)
            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(post, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(relay.annotation.stop_fusion(cast_to_int8), dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            print(cast_to_float32)
            return cast_to_float32
            # return relay.cliprelay.right_shift(relay.add(relay.cast(
            #     relay.clip(
            #         relay.round(
            #             var1 * relay.const(16.0)
            #         ), 
            #         a_min=relay.const(-127.0), 
            #         a_max=relay.const(127.0)
            #     ), dtype="int8"), relay.const(512)), 10)




In [42]:
out = rewrite(JUNOCallback(), mod['main'])

free_var %input_1: Tensor[(1, 3, 256, 256), float32];
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32];
%0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
free_var %v_param_2: Tensor[(16), float32];
%1 = nn.bias_add(%0, %v_param_2);
%2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
%3 = multiply(%2, 16f);
%4 = round(%3);
%5 = clip(%4, a_min=-127f, a_max=127f);
%6 = cast(%5, dtype="int8");
%7 = annotation.stop_fusion(%6);
%8 = cast(%7, dtype="int32");
%9 = add(%8, 512);
%10 = right_shift(%9, 10);
%11 = clip(%10, a_min=-127f, a_max=127f);
cast(%11, dtype="float32")
pass
pass


In [43]:
print(mod['main'])

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %6 = nn.conv2d(%5, %v_param_5, padding=[1, 1, 1, 1], channels=64, kernel_size=[3, 3]);
  %7 = nn.bias_add(%6, %v_param_6);
  nn.max_pool2d(%7, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0])
}


In [44]:
print(out)

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = multiply(%2, 16f);
  %4 = round(%3);
  %5 = clip(%4, a_min=-127f, a_max=127f);
  %6 = cast(%5, dtype="int8");
  %7 = annotation.stop_fusion(%6);
  %8 = cast(%7, dtype="int32");
  %9 = add(%8, 512);
  %10 = right_shift(%9, 10);
  %11 = clip(%10, a_min=-127f, a_max=127f);
  %12 = cast(%11, dtype="float32");
  %13 = nn.conv2d(%12, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %14 = nn.bias_add(%13, %v_param_4);
  %15 = nn.max_pool2d(%14, pool_size=[2, 2], strides=[

In [ ]:
print(out)

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %6 = nn.conv2d(%5, %v_param_5, padding=[1, 1, 1, 1], channels=64, kernel_size=[3, 3]);
  %7 = nn.bias_add(%6, %v_param_6);
  nn.max_pool2d(%7, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0])
}


In [ ]:
cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(mod['main'].body, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
# tvm.relay.transform.InferType(cast_to_int8)
# print(cast_to_int8)
cast_to_float32 = relay.cast(
    relay.clip(
        relay.right_shift(
            relay.add(relay.cast(cast_to_int8, dtype='int32'), relay.const(512)),
            relay.const(10)),
        a_min=-127.0, a_max=127.0), 
    dtype="float32"
)
# with 


# print(cast_to_float32)
# mod['main'] =cast_to_float32
# seq = tvm.transform.Sequential([tvm.relay.transform.InferType()])
# new_mod = seq(mod)


In [ ]:
new_part_mod = tvm.IRModule.from_expr(cast_to_float32)
# print(cast_to_float32)
# mod['main'] =cast_to_float32
seq = tvm.transform.Sequential([tvm.relay.transform.InferType()])
new_mod = seq(new_part_mod)


In [ ]:
print(cast_to_float32)

free_var %input_1: Tensor[(1, 3, 256, 256), float32];
free_var %v_param_1: Tensor[(16, 3, 3, 3), float32];
%0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
free_var %v_param_2: Tensor[(16), float32];
%1 = nn.bias_add(%0, %v_param_2);
%2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
free_var %v_param_3: Tensor[(32, 16, 3, 3), float32];
%3 = nn.conv2d(%2, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
free_var %v_param_4: Tensor[(32), float32];
%4 = nn.bias_add(%3, %v_param_4);
%5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
free_var %v_param_5: Tensor[(64, 32, 3, 3), float32];
%6 = nn.conv2d(%5, %v_param_5, padding=[1, 1, 1, 1], channels=64, kernel_size=[3, 3]);
free_var %v_param_6: Tensor[(64), float32];
%7 = nn.bias_add(%6, %v_param_6);
%8 = nn.max_pool2d(%7, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
%9 = multiply(%8, 16f);
%10 = round(%9);
%11

In [ ]:
mod['main'] = new_mod
print(mod['main'])

AssertionError: 

In [27]:
target = 'cuda'
dev = tvm.cuda()
# mod['main'] = cast_to_float32
# mod['main'] = cast_to_float32
with tvm.transform.PassContext(opt_level=0):
    lib = relay.build(out, target, params=params)

# show_graph(lib['get_graph_json'](), "quant_end_0")

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [ ]:
target = 'cuda'
dev = tvm.cuda()
# mod['main'] = cast_to_float32
with tvm.transform.PassContext(opt_level=0):
    lib = relay.build(out, target, params=params)

show_graph(lib['get_graph_json'](), "quant_0")

In [ ]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(out, target, params=params)

show_graph(lib['get_graph_json'](), "quant_3")


In [ ]:
!ls ../../tvm-slicer/src/model/unet_cuda_front_512_3_29.so

../../tvm-slicer/src/model/unet_cuda_front_512_3_29.so


In [ ]:
path = "../../tvm-slicer/src/model/unet_cuda_front_512_3_29.so"

In [ ]:
front_lib = tvm.runtime.load_module(path)

In [ ]:
print(front_lib['get_graph_json']())

{"nodes": [{"op": "null", "name": "input_0", "inputs": []}, {"op": "null", "name": "p0", "inputs": []}, {"op": "null", "name": "p1", "inputs": []}, {"op": "null", "name": "p2", "inputs": []}, {"op": "tvm_op", "name": "tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu", "attrs": {"num_outputs": "1", "num_inputs": "4", "flatten_data": "0", "func_name": "tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu", "out_layout": "", "kernel_layout": "OIHW", "data_layout": "NCHW", "hash": "3dbb7c2534747700"}, "inputs": [[0, 0, 0], [1, 0, 0], [2, 0, 0], [3, 0, 0]]}, {"op": "null", "name": "p3", "inputs": []}, {"op": "null", "name": "p4", "inputs": []}, {"op": "null", "name": "p5", "inputs": []}, {"op": "null", "name": "p6", "inputs": []}, {"op": "tvm_op", "name": "tvmgen_default_fused_nn_conv2d_add_multiply_add", "attrs": {"num_outputs": "1", "num_inputs": "5", "flatten_data": "0", "func_name": "tvmgen_default_fused_nn_conv2d_add_multiply_add", "out_layout": "", "kernel_layout": "OIHW", "data_l

In [ ]:
from tvm.contrib import graph_executor


In [ ]:
front_model = graph_executor.GraphModule(front_lib['default'](tvm.cuda()))

In [ ]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("unet", 512))


In [ ]:
input_data = np.random.normal(0,1,(1,512,512,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

In [ ]:
target = 'cuda'
dev = tvm.cuda()
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

In [ ]:
print(mod.astext())

#[version = "0.0.5"]
def @main(%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3

In [ ]:
from SlicingMachine import TVMSlicer


In [ ]:
graph_json_raw

'{\n  "nodes": [\n    {\n      "op": "null", \n      "name": "input_1", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "p0", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "p1", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "p2", \n      "inputs": []\n    }, \n    {\n      "op": "tvm_op", \n      "name": "tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu", \n      "attrs": {\n        "num_outputs": "1", \n        "num_inputs": "4", \n        "flatten_data": "0", \n        "func_name": "tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu", \n        "out_layout": "", \n        "kernel_layout": "OIHW", \n        "data_layout": "NCHW", \n        "hash": "3dbb7c2534747700"\n      }, \n      "inputs": [\n        [\n          0, \n          0, \n          0\n        ], \n        [\n          1, \n          0, \n          0\n        ], \n        [\n          2, \n          0, \n          0\n        ], \n    

In [ ]:
graph_json_raw = lib['get_graph_json']()
tvm_slicer = TVMSlicer(graph_json_raw)


In [ ]:
graph_json_front_info = tvm_slicer.slice_graph(0, 29)
graph_json_back_info = tvm_slicer.slice_graph(29, 111)


In [ ]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build_graph(mod, target=target, target_host=None, params=params, mod_name="default", graph_config=json.dumps(graph_json_front_info))
            

In [ ]:
lib.ir_mod

#[version = "0.0.5"]
def @main(%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3